In [ ]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns

In [ ]:
def TimeDiff(df, frameSize): #adds time difference between packets as  a parameter
    count = 1
    classLabel = df.loc[1,"ClassLabel"]
    time_diff = [0]
    rows = (df.shape)[0]
    for i in range(1,rows):
        if((df.loc[i+1,"ClassLabel"] == classLabel) and (count < frameSize)):
            diff = datetime.datetime.strptime(df.iloc[i,0].split(' ')[3],"%H:%M:%S") - datetime.datetime.strptime(df.iloc[i-1,0].split(' ')[3],"%H:%M:%S")
            time_diff.append(diff.total_seconds())
            count = count + 1
        else:
            time_diff.append(0)
            classLabel = df.loc[i+1,"ClassLabel"]
            count = 1
    df["SNMP Parameters"] = time_diff
    df.rename(index=str, columns={"SNMP Parameters": "Time Difference"})

In [ ]:
def difference(df,cols):
    classLabel = df.loc[1,"ClassLabel"]
    diffList = []

    for j in cols:
        diffList.append([df.loc[1,j]])

    rows = (df.shape)[0]
    for i in range(2,rows+1):
        if((df.loc[i,"ClassLabel"] == classLabel)):
            for j in range(len(cols)):
                diffList[j].append(df.loc[i,cols[j]] - df.loc[i-1,cols[j]])
        else:
            for j in range(len(cols)):
                diffList[j].append(df.loc[i,cols[j]])
            classLabel = df.loc[i,"ClassLabel"]

    for j in range(len(cols)):
        df[cols[j]] = diffList[j]

In [ ]:
def makeNewDataFrame(df, frameSize):
    i = 0
    listOfRowVals = []
    rows = (df.shape)[0]
    while(i+frameSize-1<rows):
        if(df.iloc[i,-1] == df.iloc[i+frameSize-1,-1]):
            rowVals = []
            for j in range(frameSize):
                rowVals = rowVals + list(df.iloc[i+j,:-1])
            rowVals.append(df.iloc[i,-1])
            i = i+frameSize
            listOfRowVals.append(rowVals)
        else:
            j = 1
            while(df.iloc[i,-1] == df.iloc[i+j,-1]):
                j=j+1
            i = i+j
    new_df = pd.DataFrame(listOfRowVals)
    return new_df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## LOAD DATA

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Flooded SNMP packets detection/SNMP_Dataset.csv')
#To drop iso values
df = df[1:]

#Coding Class Types into integers
# multiclass
labelDict = {'ICMP_Normal' : 0, 'UDP_Normal' : 1, 'LAND_Normal' : 2,
            'TCPSYN_Normal' : 3, 'TCPSYNACK_Normal' : 4, 'ICMP_Flood' : 5,
            'UDP_Flood' : 6, 'LAND_Flood' : 7, 'TCPSYN_Flood' : 8,
             'TCPSYNACK_Flood' : 9}
# binary
# labelDict = {'ICMP_Normal' : 0, 'UDP_Normal' : 0, 'LAND_Normal' : 0,
#             'TCPSYN_Normal' : 0, 'TCPSYNACK_Normal' : 0, 'ICMP_Flood' : 1,
#             'UDP_Flood' : 1, 'LAND_Flood' : 1, 'TCPSYN_Flood' : 1,
#              'TCPSYNACK_Flood' : 1}

limit = len(df['ClassLabel']) + 1
#df['ClassLabel'] = df['ClassLabel'].replace(['ICMP_Normal','UDP_Normal','LAND_Normal','TCPSYN_Normal','TCPSYNACK_Normal','ICMP_Flood','UDP_Flood','LAND_Flood','TCPSYN_Flood','TCPSYNACK_Flood'],[0,1,2,3,4,5,6,7,8,9])
for i in range(1, limit):
    df['ClassLabel'][i] = labelDict[df['ClassLabel'][i]]

print(df.shape)
df.head()

(2421, 66)


,SNMP Parameters,icmp_AddrMaskReps,icmp_InAddrMasks,icmp_InDestUnreachs,icmp_InEchosReps,icmp_InEchos,icmp_InErrors,icmp_InMsgs,icmp_InParmProbs,icmp_InRedirects,...,tcp_PassiveOpens,tcp_RetransSegs,tcp_RtoAlgorithm,tcp_RtoMax,tcp_RtoMin,udp_InDatagrams,udp_InErrors,udp_NoPorts,udp_OutDatagrams,ClassLabel
1,Mon Jul 31 15:49:58 UTC 2017,0,0,0,16,0,0,17,0,0,...,1,0,1,120000,200,148,0,1,149,0
2,Mon Jul 31 15:50:31 UTC 2017,0,0,0,111,0,0,112,0,0,...,2,0,1,120000,200,445,0,1,446,0
3,Mon Jul 31 15:51:10 UTC 2017,0,0,0,229,0,0,230,0,0,...,2,0,1,120000,200,741,0,1,742,0
4,Mon Jul 31 15:51:44 UTC 2017,0,0,0,0,0,0,332,0,0,...,2,0,1,120000,200,1037,0,1,1038,0
5,Mon Jul 31 15:52:24 UTC 2017,0,0,0,442,0,0,443,0,0,...,2,0,1,120000,200,1333,0,1,1334,0


In [ ]:
df.info()

In [ ]:
df.describe()

columns with only one value are present

## Preprocess

In [ ]:
frameSize = 6
TimeDiff(df, frameSize)
df

,SNMP Parameters,icmp_AddrMaskReps,icmp_InAddrMasks,icmp_InDestUnreachs,icmp_InEchosReps,icmp_InEchos,icmp_InErrors,icmp_InMsgs,icmp_InParmProbs,icmp_InRedirects,...,tcp_PassiveOpens,tcp_RetransSegs,tcp_RtoAlgorithm,tcp_RtoMax,tcp_RtoMin,udp_InDatagrams,udp_InErrors,udp_NoPorts,udp_OutDatagrams,ClassLabel
1,0.0,0,0,0,16,0,0,17,0,0,...,1,0,1,120000,200,148,0,1,149,0
2,33.0,0,0,0,111,0,0,112,0,0,...,2,0,1,120000,200,445,0,1,446,0
3,39.0,0,0,0,229,0,0,230,0,0,...,2,0,1,120000,200,741,0,1,742,0
4,34.0,0,0,0,0,0,0,332,0,0,...,2,0,1,120000,200,1037,0,1,1038,0
5,40.0,0,0,0,442,0,0,443,0,0,...,2,0,1,120000,200,1333,0,1,1334,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2417,71.0,0,0,0,0,0,0,1,0,0,...,2,15,1,120000,200,72673,0,571583939,72674,6
2418,0.0,0,0,0,0,0,0,1,0,0,...,2,15,1,120000,200,72969,0,573650651,72970,6
2419,79.0,0,0,0,0,0,0,1,0,0,...,2,15,1,120000,200,73265,0,576077076,73266,6
2420,83.0,0,0,0,0,0,0,1,0,0,...,2,15,1,120000,200,73561,0,577961719,73562,6


In [ ]:
df = df.astype(int)
print('cols with 0 std:', len(list((df.loc[:, df.std() == 0]).columns)))
print((df.loc[:, df.std() == 0]).columns)
print(df.shape)

cols with 0 std: 36
Index(['icmp_AddrMaskReps', 'icmp_InAddrMasks', 'icmp_InDestUnreachs',
       'icmp_InEchos', 'icmp_InParmProbs', 'icmp_InRedirects',
       'icmp_InSrcQuenchs', 'icmp_InTimestampReps', 'icmp_OutAddrMaskReps',
       'icmp_OutAddrMasks', 'icmp_OutDestUnreachs', 'icmp_OutEchos',
       'icmp_OutMsgs', 'icmp_OutParmProbs', 'icmp_OutRedirects',
       'icmp_OutSrcQuenchs', 'icmp_OutTimestampsReps', 'ip_DefaultTTL',
       'ip_Forwarding', 'ip_ForwDatagrams', 'ip_FragFails', 'ip_InDiscard',
       'ip_InHdrErrors', 'ip_InUnknownProtos', 'ip_OutDiscards',
       'ip_OutNoRoutes', 'ip_ReasmFails', 'ip_ReasmTimeout',
       'ip_RoutingDiscards', 'tcp_AttemptFails', 'tcp_InErrs', 'tcp_MaxConn',
       'tcp_RtoAlgorithm', 'tcp_RtoMax', 'tcp_RtoMin', 'udp_InErrors'],
      dtype='object')
(2421, 66)


In [ ]:
#To drop columns with same values throughout -> 36 columns got dropped
#Dimentionality Reduction

df = df.drop(df.std()[(df.std() == 0)].index, axis=1)
print(len(list(df.columns)))
print(df.columns)

30
Index(['SNMP Parameters', 'icmp_InEchosReps', 'icmp_InErrors', 'icmp_InMsgs',
       'icmp_InTimeExcds', 'icmp_InTimestamps', 'icmp_OutEchoReps',
       'icmp_OutErrors', 'icmp_OutTimeExcds', 'icmp_OutTimestamps',
       'ip_FragCreates', 'ip_FragOKs', 'ip_InAddrErrors', 'ip_InDelivers',
       'ip_InReceives', 'ip_OutRequests', 'ip_ReasmOKs', 'ip_ReasmReqds',
       'tcp_ActiveOpens', 'tcp_CurrEstablish', 'tcp_EstabResets', 'tcp_InSegs',
       'tcp_OutRsts', 'tcp_OutSegs', 'tcp_PassiveOpens', 'tcp_RetransSegs',
       'udp_InDatagrams', 'udp_NoPorts', 'udp_OutDatagrams', 'ClassLabel'],
      dtype='object')


In [ ]:
colsToFindDifference=['icmp_InEchosReps','icmp_InMsgs','icmp_OutErrors','icmp_OutTimestamps','ip_InDelivers','ip_InReceives','ip_OutRequests','udp_InDatagrams','udp_OutDatagrams']
difference(df,colsToFindDifference)

# print(len(list(df.columns)))
# print(df.shape)

In [ ]:
df_ = makeNewDataFrame(df, frameSize)
#df_.to_csv('dont run this /content/drive/My Drive/Colab Notebooks/Flooded SNMP packets detection/transormed_SNMPdata_binary.csv')
#df_.to_csv('dont run this /content/drive/My Drive/Colab Notebooks/Flooded SNMP packets detection/transormed_SNMPdata_multiclass.csv')

# print(df_.shape)
# df_.head()


(403, 175)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174
0,0,16,0,17,1,0,0,17,1,16,0,0,2,194,196,229,0,0,23,1,0,178,0,213,1,0,148,1,149,33,95,0,95,1,0,0,95,1,111,0,...,2,0,273,0,296,2,0,296,1,296,32,106,0,106,1,0,0,123,1,123,0,0,2,392,392,391,0,0,23,2,0,273,0,296,2,0,296,2,296,0
1,0,108,0,108,1,0,0,91,1,91,0,0,2,423,423,423,0,0,23,2,0,273,0,296,2,0,296,2,296,30,110,0,110,1,0,0,125,1,125,0,...,2,0,273,0,296,2,0,296,2,296,32,118,0,118,1,0,0,134,1,118,0,0,2,392,392,392,0,0,24,2,0,273,0,296,2,0,296,2,296,0
2,0,98,0,98,1,0,0,82,1,97,0,0,2,414,414,414,0,0,24,2,0,273,0,296,2,0,296,2,296,33,124,0,124,1,0,0,124,1,109,0,...,2,0,299,0,316,2,0,296,2,296,37,125,0,125,1,0,0,125,1,110,0,0,2,407,407,407,0,0,24,2,0,299,0,316,2,0,296,2,296,0
3,0,93,0,93,1,0,0,93,1,108,0,0,2,401,401,401,0,0,24,2,0,299,0,316,2,0,296,2,296,37,120,0,120,1,0,0,120,1,105,0,...,2,0,299,0,316,2,0,296,2,296,40,113,0,113,1,0,0,-2386,0,-2401,0,0,2,417,417,417,0,0,24,2,0,299,0,316,2,0,296,2,296,0
4,0,104,0,104,1,0,0,2618,1,2617,0,0,2,390,390,390,0,0,24,2,0,299,0,316,2,0,296,2,296,41,110,0,110,1,0,0,95,1,95,0,...,2,0,299,0,316,2,0,296,2,296,40,3139,0,103,1,0,0,88,1,103,0,0,2,418,418,418,0,0,24,2,0,299,0,316,2,0,296,2,296,0
